# Caching

- Author: [Joseph](https://github.com/XaviereKU)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

`LangChain` provides optional caching layer for LLMs.

This is useful for two reasons:
- When requesting the same completions multiple times, it can **reduce the number of API calls** to the LLM provider and thus save costs.
- By **reduing the number of API calls** to the LLM provider, it can **improve the running time of the application.**

In this tutorial, we will use gpt-3.5-turbo OpenAI API and utilize two kinds of cache, **InMemoryCache** and **SQLite Cache** .  
At end of each section we will compare wall times between before and after caching.

Optionally, we will use local LLM served with VLLM.

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [InMemoryCache](#in-memory-cache)
- [SQlite Cache](#sqlite-cache)
- [(Optional) With local model](#optional-with-local-model)
- [(Optional) InMemoryCache + Local LLM](#optional-inmemorycache--local-llm)
- [(Optional) SQLite Cache + Local LLM](#optional-sqlite-cache--local-llm)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
        # "vllm", # this is for optional section
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "Your API KEY",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Caching",
    }
)

Environment variables have been set successfully.


In [ ]:
# Alternatively, one can set environmental variables with load_dotenv
from dotenv import load_dotenv



load_dotenv()

False

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Create model
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Generate prompt
prompt = PromptTemplate.from_template(
    "Sumarize about the {country} in about 200 characters"
)

# Create chain
chain = prompt | llm

In [7]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

c:\Users\harry\AppData\Local\pypoetry\Cache\virtualenvs\langchain-opentutorial-6kFp1u2S-py3.11\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=70f8b79c-2576-4959-8387-7bb865d0d251,id=70f8b79c-2576-4959-8387-7bb865d0d251; trace=70f8b79c-2576-4959-8387-7bb865d0d251,id=65861779-ce78-4325-9650-9f3351d71323; trace=70f8b79c-2576-4959-8387-7bb865d0d251,id=b6724e0d-c4ae-4fa4-9470-a942d4969426


South Korea is a highly developed country in East Asia known for its advanced technology, vibrant culture, and booming economy. It is home to major tech companies like Samsung and LG, and boasts a rich history and stunning natural landscapes.
CPU times: total: 109 ms
Wall time: 1.01 s


Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=70f8b79c-2576-4959-8387-7bb865d0d251,id=70f8b79c-2576-4959-8387-7bb865d0d251; trace=70f8b79c-2576-4959-8387-7bb865d0d251,id=b6724e0d-c4ae-4fa4-9470-a942d4969426
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=b5b95852-b438-4dd0-991a-44933b7478dc,id=b5b95852-b438-4dd0-991a-44933b7478dc; trace=b5b95852-b438-4dd0-991a-44933b7478dc,id=528f3e22-1dc2-43b2-ac5b-e833398fff8e; trace=b5b95852-b438-4dd0-991a-44933b7478dc,id=f65f6a12-9817-48c2-9ad2-4caf13ac71a0
Failed to multipart ingest runs: langs

## InMemoryCache
First, cache the answer to the same question using InMemoryCache.

In [8]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# Set InMemoryCache
set_llm_cache(InMemoryCache())

In [9]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea, located in East Asia, is known for its advanced technology, dynamic culture, and booming economy. It is home to K-pop, kimchi, and beautiful landscapes. The country has a rich history, delicious cuisine, and a strong emphasis on education.
CPU times: total: 15.6 ms
Wall time: 950 ms


Now we invoke the chain with the same question.

In [10]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea, located in East Asia, is known for its advanced technology, dynamic culture, and booming economy. It is home to K-pop, kimchi, and beautiful landscapes. The country has a rich history, delicious cuisine, and a strong emphasis on education.
CPU times: total: 0 ns
Wall time: 3.15 ms


Note that if we set InMemoryCache again, the cache will be lost and the wall time will increase

In [11]:
set_llm_cache(InMemoryCache())

In [12]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a vibrant and modern country in East Asia known for its technological advancements, rich cultural heritage, and delicious cuisine. It is home to bustling cities like Seoul, beautiful natural landscapes, and a strong economy driven by industries such as technology, automotive, and entertainment.
CPU times: total: 0 ns
Wall time: 988 ms


## SQLite Cache
Now, we cache the answer to the same question by using SQLite cache.

In [13]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia known for its booming economy, vibrant culture, and delicious cuisine. It is home to global companies like Samsung and Hyundai, as well as historical sites like Gyeongbokgung Palace and the DMZ. The country has a rich history and is a popular destination for tourists seeking a mix of traditional and modern experiences.
CPU times: total: 15.6 ms
Wall time: 1.12 s


Now we invoke the chain with the same question.

In [15]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia known for its booming economy, vibrant culture, and delicious cuisine. It is home to global companies like Samsung and Hyundai, as well as historical sites like Gyeongbokgung Palace and the DMZ. The country has a rich history and is a popular destination for tourists seeking a mix of traditional and modern experiences.
CPU times: total: 391 ms
Wall time: 398 ms


Note that if we use SQLite Cache, setting caching again does not delete store cache

In [16]:
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [17]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia known for its booming economy, vibrant culture, and delicious cuisine. It is home to global companies like Samsung and Hyundai, as well as historical sites like Gyeongbokgung Palace and the DMZ. The country has a rich history and is a popular destination for tourists seeking a mix of traditional and modern experiences.
CPU times: total: 0 ns
Wall time: 4 ms


## (Optional) With local model
In this optional section, we utilize `docker` to serve local LLM model.
Note that this used miniconda to set environment easily.

### Device & Serving information
- CPU : AMD 5600X
- OS : Windows 10 Pro
- RAM : 32 Gb
- GPU : Nividia 3080Ti, 12GB VRAM
- CUDA : 12.6
- Driver Version : 560.94
- docker image : nvidia/cuda:12.4.1-cudnn-devel-ubuntu20.04
- model : Qwen/Qwen2.5-0.5B-Instruct
- Python version : 3.10
- docker run script :
    ```
    docker run -itd --name vllm --gpus all --entrypoint /bin/bash -p 6001:8888 nvidia/cuda:12.4.1-cudnn-devel-ubuntu20.04
    ```
- vllm serving script : 
    ```
    python3 -m vllm.entrypoints.openai.api_server --model='Qwen/Qwen2.5-0.5B-Instruct' --served-model-name 'qwen-2.5' --port 8888 --host 0.0.0.0 --gpu-memory-utilization 0.80 --max-model-len 4096 --swap-space 1 --dtype bfloat16 --tensor-parallel-size 1 
    ```

In [18]:
from langchain_community.llms import VLLMOpenAI

# create model using OpenAI compatible class VLLMOpenAI
llm = VLLMOpenAI(
    model="qwen-2.5", openai_api_key="EMPTY", openai_api_base="http://localhost:6001/v1"
)

# Generate prompt
prompt = PromptTemplate.from_template(
    "Sumarize about the {country} in about 200 characters"
)

# Create chain
chain = prompt | llm

## (Optional) InMemoryCache + Local LLM
Same InMemoryCache section above, we set InMemoryCache.

In [19]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# Set InMemoryCache
set_llm_cache(InMemoryCache())

Invoke chain with local LLM, do note that we print **response** not **response.content**

In [20]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.
South Korea is a country in East Asia, with a population of approximately 55.2 million as of 2023. It borders North Korea to the east, Japan to the northeast, and China to the southeast. The country is known for its advanced technology, leading industries, and significant contributions to South Korean culture. It is often referred to as the "Globe and a Couple" due to its diverse landscapes, rich history, and frontiers with neighboring countries. South Korea's economy is growing, with a strong technological sector and a strong economy, making it a significant player on the global stage. Overall, South Korea is a significant global player, with a rich history, advanced technology, and a cultural influence. With its advanced technology and unique culture, South Korea is a fascinating country to explore. Its diverse landscapes, rich history, and remarkable economic performance have made it a popular destination for travelers. South Korea's contribution to the global economy and its stro

Now we invoke chain again, with the same question.

In [21]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.
South Korea is a country in East Asia, with a population of approximately 55.2 million as of 2023. It borders North Korea to the east, Japan to the northeast, and China to the southeast. The country is known for its advanced technology, leading industries, and significant contributions to South Korean culture. It is often referred to as the "Globe and a Couple" due to its diverse landscapes, rich history, and frontiers with neighboring countries. South Korea's economy is growing, with a strong technological sector and a strong economy, making it a significant player on the global stage. Overall, South Korea is a significant global player, with a rich history, advanced technology, and a cultural influence. With its advanced technology and unique culture, South Korea is a fascinating country to explore. Its diverse landscapes, rich history, and remarkable economic performance have made it a popular destination for travelers. South Korea's contribution to the global economy and its stro

## (Optional) SQLite Cache + Local LLM
Same as SQLite Cache section above, set SQLite Cache.  
Note that we set db name to be **vllm_cache.db** to distinguish from the cache used in SQLite Cache section.

In [23]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/vllm_cache.db"))

Invoke chain with local LLM, again, note that we print **response** not **response.content**

In [24]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.

South Korea, a nation that prides itself on its history, culture, and natural beauty. Known for its bustling cityscapes, scenic valleys, and delicious cuisine. A major player in South East Asia and a global hub for technology, fashion, and entertainment. Home to industries like electronics, automotive, and media. With a strong economy, South Korea is among the top economies in the world, known for its efficient and inclusive societies. A country that has been a significant player in global politics for decades. The country is also home to many influential figures like Kim Jong-un and Kim Jong-un, who have led North Korea and the country’s military. Known for its national sports, including football (soccer), baseball, and gymnastics. South Korea is also home to many museums, art galleries, and historical sites, showcasing the country’s rich cultural heritage. The country is a leader in technology, with many leading companies based in the South Korean capital, Seoul. The South Korean 

Now we invoke chain again, with the same question.

In [25]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response)

.

South Korea, a nation that prides itself on its history, culture, and natural beauty. Known for its bustling cityscapes, scenic valleys, and delicious cuisine. A major player in South East Asia and a global hub for technology, fashion, and entertainment. Home to industries like electronics, automotive, and media. With a strong economy, South Korea is among the top economies in the world, known for its efficient and inclusive societies. A country that has been a significant player in global politics for decades. The country is also home to many influential figures like Kim Jong-un and Kim Jong-un, who have led North Korea and the country’s military. Known for its national sports, including football (soccer), baseball, and gymnastics. South Korea is also home to many museums, art galleries, and historical sites, showcasing the country’s rich cultural heritage. The country is a leader in technology, with many leading companies based in the South Korean capital, Seoul. The South Korean 